In [1]:
import pandas as pd
import plotly.express as px 
from urllib.request import urlopen
import json
import janitor
import plotly.io as pio
from json import loads, dumps
pio.renderers.default='notebook'

In [ ]:
asthmaca = pd.read_csv('asthma_redownload_utf8.csv')
asthmaca = asthmaca.clean_names()
asthmaca

In [ ]:
asthmaca = asthmaca.query('county != \'California\'')
asthmaca = asthmaca.query('year != 2015')
asthmaca = asthmaca.query('year != 2020')
asthmaca

In [4]:
asthmaca = asthmaca[asthmaca['strata'] == 'Race/ethnicity'].copy()

In [5]:
with open('California_County_Boundaries.json') as jsonfile:
    geojson = json.load(jsonfile)
geodf = pd.json_normalize(geojson, record_path='features')
fipmap = geodf.set_index('properties.CountyName')['properties.County_FIPS_ID'].to_dict()
asthmaca['fip'] = asthmaca['county'].map(fipmap)
asthmaca['fip'] = asthmaca['fip'].astype(str)

In [ ]:
fig = px.choropleth(
    asthmaca.query('year == 2019'),
    geojson=geojson,
    featureidkey='properties.County_FIPS_ID',
    locations='fip',
    color='age_adjusted_hospitalization_rate',
    hover_data=['county'],
    animation_frame='strata_name',
    width=1200,
    labels={
        'age_adjusted_hospitalization_rate':'Age Adjusted Hospitalization Rate (per 10,000)'
    },
    color_continuous_scale='YlGn'
)

fig.update_geos(fitbounds="locations", visible=False)

fig.show()

In [ ]:
fig = px.choropleth(
    asthmaca.query('year == 2019'),
    geojson=geojson,
    featureidkey='properties.County_FIPS_ID',
    locations='fip',
    color='age_adjusted_hospitalization_rate',
    range_color=(0, 25),
    hover_data=['county'],
    animation_frame='strata_name',
    labels={
        'age_adjusted_hospitalization_rate':'Age Adjusted Hospitalization Rate (per 10,000)'
    },
    color_continuous_scale='YlGn'
)

fig.update_geos(fitbounds="locations", visible=False)

fig.show()

In [10]:
fig.write_html('StrataAnalysisGeo_CorrectWidth.html')